In [1]:
from sympy import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#insert filepath to "HR_data.csv"
filename = "filepath/HR_data.csv"
df = pd.read_csv(filename)
df = df.iloc[:, 1:]
df 

,HR_Mean,HR_Median,HR_std,HR_Min,HR_Max,HR_AUC,Round,Phase,Individual,Puzzler,Frustrated,Cohort
0,77.965186,78.000,3.345290,73.23,83.37,22924.945,round_3,phase3,1,1,1,D1_1
1,70.981097,70.570,2.517879,67.12,78.22,21930.400,round_3,phase2,1,1,5,D1_1
2,73.371959,73.360,3.259569,67.88,80.22,21647.085,round_3,phase1,1,1,0,D1_1
3,78.916822,77.880,4.054595,72.32,84.92,25258.905,round_2,phase3,1,1,1,D1_1
4,77.322226,74.550,6.047603,70.52,90.15,23890.565,round_2,phase2,1,1,5,D1_1
...,...,...,...,...,...,...,...,...,...,...,...,...
163,73.594539,72.380,9.474556,57.43,93.53,21482.985,round_4,phase2,14,0,8,D1_2
164,57.839897,54.130,6.796647,52.97,74.14,16825.740,round_4,phase1,14,0,0,D1_2
165,64.237295,65.195,3.589241,58.97,72.63,18691.065,round_1,phase3,14,0,1,D1_2
166,70.834320,70.440,2.391160,66.65,76.07,20753.005,round_1,phase2,14,0,4,D1_2


In [3]:
frustration_counts = df['Frustrated'].value_counts()
print(frustration_counts)

Frustrated
1    45
2    31
0    29
4    23
3    16
5    14
6     5
8     3
7     2
Name: count, dtype: int64


Applying hold-out CV

In [4]:
from sklearn.model_selection import train_test_split

# Splitting individuals to prevent data leakage
unique_individuals = df['Individual'].unique()
train_inds, test_inds = train_test_split(unique_individuals, test_size=0.3, random_state=42) # 30% test, 70% train

# Splitting the dataset based on the individuals
train_df = df[df['Individual'].isin(train_inds)]
test_df = df[df['Individual'].isin(test_inds)]

print(f"Training set size: {train_df.shape}")
print(f"Testing set size: {test_df.shape}")

Training set size: (108, 12)
Testing set size: (60, 12)


In [5]:
train_df

,HR_Mean,HR_Median,HR_std,HR_Min,HR_Max,HR_AUC,Round,Phase,Individual,Puzzler,Frustrated,Cohort
12,70.088074,70.060,2.474921,66.03,75.17,20672.675,round_3,phase3,2,0,2,D1_1
13,74.946026,75.820,3.761099,65.42,81.50,23305.900,round_3,phase2,2,0,4,D1_1
14,78.225679,73.080,10.984611,70.92,109.35,22364.180,round_3,phase1,2,0,1,D1_1
15,72.933270,71.320,4.667087,68.27,84.32,22897.795,round_2,phase3,2,0,2,D1_1
16,71.929740,72.175,4.514956,62.78,80.98,22077.910,round_2,phase2,2,0,5,D1_1
...,...,...,...,...,...,...,...,...,...,...,...,...
163,73.594539,72.380,9.474556,57.43,93.53,21482.985,round_4,phase2,14,0,8,D1_2
164,57.839897,54.130,6.796647,52.97,74.14,16825.740,round_4,phase1,14,0,0,D1_2
165,64.237295,65.195,3.589241,58.97,72.63,18691.065,round_1,phase3,14,0,1,D1_2
166,70.834320,70.440,2.391160,66.65,76.07,20753.005,round_1,phase2,14,0,4,D1_2


Handling data

In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))

In [7]:
X_train = sc.fit_transform(train_df.iloc[:, :6])
y_train = train_df["Frustrated"].values

In [8]:
X_test = sc.fit_transform(test_df.iloc[:, :6])
y_test = test_df["Frustrated"].values

In [9]:
X_train.shape, y_train.shape

((108, 6), (108,))

In [10]:
X_test.shape, y_test.shape

((60, 6), (60,))

Training a Logistic Regression Classifier

In [11]:
from sklearn.linear_model import LogisticRegression

n_runs = 100
LR_accuracies = []

for i in range(n_runs):
    LR = LogisticRegression()
    LR.fit(X_train, y_train)
    LR_accuracies.append(LR.score(X_test,y_test))

print(f'Average accuracy over {n_runs} runs: {np.mean(LR_accuracies)}')


Average accuracy over 100 runs: 0.15


Training an Artificial Neural Network (ANN)

In [12]:
from sklearn.neural_network import MLPClassifier

n_runs = 100
ANN_accuracies = []

for i in range(n_runs):
    ANN = MLPClassifier(solver='lbfgs', alpha=1e-5,
                        hidden_layer_sizes=(5), random_state=i) 
    ANN.fit(X_train, y_train)
    ANN_accuracies.append(ANN.score(X_test,y_test))

print(f'Average accuracy over {n_runs} runs: {np.mean(ANN_accuracies)}')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Average accuracy over 100 runs: 0.14783333333333334


Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

n_runs = 100
RF_accuracies = []

estimators = 50

for i in range(n_runs):
    RF = RandomForestClassifier(random_state=i,n_estimators=estimators)
    RF.fit(X_train, y_train)
    RF_accuracies.append(RF.score(X_test,y_test))
    
print(f'Average accuracy over {n_runs} runs: {np.mean(RF_accuracies)}')

Average accuracy over 100 runs: 0.10149999999999998


Exporting accuracies for analysis

In [14]:
# Export accuracies to CSV for further analysis in R
df_export = pd.DataFrame({
    'ANN_accuracies': ANN_accuracies,
    'RF_accuracies': RF_accuracies
})
#insert filepath to "model_accuracies.csv"
df_export.to_csv('filepath/model_accuracies.csv', index=False)